In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, precision_score, f1_score, recall_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import MultiTaskLasso
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LassoLars
from math import sqrt
from sklearn.metrics import median_absolute_error, explained_variance_score,max_error,mean_absolute_error,mean_poisson_deviance,mean_gamma_deviance

In [2]:
df=pd.read_csv('./input/train.csv',encoding='latin-1')

In [3]:
df = df.set_index('id', drop=True)

In [4]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.12,Premium,H,SI1,61.6,59.0,6.67,6.63,4.10,5363
1,1.14,Very Good,E,SI2,60.0,54.0,6.74,6.97,4.11,5593
2,0.90,Very Good,D,SI2,60.3,63.0,6.12,6.22,3.72,3534
3,0.71,Ideal,E,VS1,61.9,54.0,5.74,5.76,3.56,3212
4,0.34,Very Good,F,SI2,60.0,62.0,4.51,4.55,2.72,447


In [5]:
print(df.info, df.dtypes)

<bound method DataFrame.info of        carat        cut color clarity  depth  table     x     y     z  price
id                                                                          
0       1.12    Premium     H     SI1   61.6   59.0  6.67  6.63  4.10   5363
1       1.14  Very Good     E     SI2   60.0   54.0  6.74  6.97  4.11   5593
2       0.90  Very Good     D     SI2   60.3   63.0  6.12  6.22  3.72   3534
3       0.71      Ideal     E     VS1   61.9   54.0  5.74  5.76  3.56   3212
4       0.34  Very Good     F     SI2   60.0   62.0  4.51  4.55  2.72    447
...      ...        ...   ...     ...    ...    ...   ...   ...   ...    ...
40450   0.90       Good     D     SI1   63.8   58.0  6.13  6.09  3.90   4120
40451   0.35      Ideal     D    VVS1   62.0   54.0  4.56  4.57  2.83   1266
40452   0.90       Good     G     SI2   58.4   55.0  6.34  6.39  3.72   3269
40453   1.21       Good     H     SI1   63.6   56.0  6.63  6.68  4.23   5893
40454   0.41      Ideal     H    VVS1   61.4

In [6]:
df.isna().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [7]:
df.describe()

,carat,depth,table,x,y,z,price
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,0.798121,61.745132,57.451892,5.731217,5.735248,3.538685,3938.101125
std,0.474880,1.434259,2.225893,1.123364,1.152859,0.710579,3992.645108
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,326.000000
25%,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,948.000000
50%,0.700000,61.800000,57.000000,5.700000,5.710000,3.520000,2401.000000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.040000,5333.000000
max,5.010000,79.000000,79.000000,10.740000,58.900000,31.800000,18818.000000


In [8]:
'''#Checking on price-to-size relation
df['size']=round(df['x']*df['y']*df['z'], 1)
df['prize-to-size']=round(df['price']/df['size'], 1)
df.head()'''

"#Checking on price-to-size relation\ndf['size']=round(df['x']*df['y']*df['z'], 1)\ndf['prize-to-size']=round(df['price']/df['size'], 1)\ndf.head()"

In [9]:
print('CUT\n', df.cut.value_counts(),'\n\nCOLOR\n', df.color.value_counts(), '\n\nCLARITY\n', df.clarity.value_counts())

CUT
 Ideal        16250
Premium      10342
Very Good     9026
Good          3639
Fair          1198
Name: cut, dtype: int64 

COLOR
 G    8565
E    7344
F    7162
H    6147
D    5063
I    4052
J    2122
Name: color, dtype: int64 

CLARITY
 SI1     9806
VS2     9204
SI2     6865
VS1     6099
VVS2    3778
VVS1    2776
IF      1365
I1       562
Name: clarity, dtype: int64


In [10]:
labels_cut= {'Fair': 4,'Good': 3, 'Very Good': 2, 'Premium': 1, 'Ideal': 0}
labels_color= {'J': 6,'I': 5,'H': 4,'G': 3,'F': 2,'E': 1,'D': 0}
labels_clarity= {'I1': 7,'SI2': 6,'SI1': 5,'VS2': 4,'VS1': 3,'VVS2': 2,'VVS1': 1,'IF': 0}

In [11]:
df1 = df.replace({'cut': labels_cut})
df1 = df1.replace({'color': labels_color})
df1 = df1.replace({'clarity': labels_clarity})

In [12]:
train_dataset = df1
train_dataset

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.12,1,4,5,61.6,59.0,6.67,6.63,4.10,5363
1,1.14,2,1,6,60.0,54.0,6.74,6.97,4.11,5593
2,0.90,2,0,6,60.3,63.0,6.12,6.22,3.72,3534
3,0.71,0,1,3,61.9,54.0,5.74,5.76,3.56,3212
4,0.34,2,2,6,60.0,62.0,4.51,4.55,2.72,447
...,...,...,...,...,...,...,...,...,...,...
40450,0.90,3,0,5,63.8,58.0,6.13,6.09,3.90,4120
40451,0.35,0,0,1,62.0,54.0,4.56,4.57,2.83,1266
40452,0.90,3,3,6,58.4,55.0,6.34,6.39,3.72,3269


In [13]:
round(train_dataset.describe(),1)

,carat,cut,color,clarity,depth,table,x,y,z,price
count,40455.0,40455.0,40455.0,40455.0,40455.0,40455.0,40455.0,40455.0,40455.0,40455.0
mean,0.8,1.1,2.6,3.9,61.7,57.5,5.7,5.7,3.5,3938.1
std,0.5,1.1,1.7,1.7,1.4,2.2,1.1,1.2,0.7,3992.6
min,0.2,0.0,0.0,0.0,43.0,43.0,0.0,0.0,0.0,326.0
25%,0.4,0.0,1.0,3.0,61.0,56.0,4.7,4.7,2.9,948.0
50%,0.7,1.0,3.0,4.0,61.8,57.0,5.7,5.7,3.5,2401.0
75%,1.0,2.0,4.0,5.0,62.5,59.0,6.5,6.5,4.0,5333.0
max,5.0,4.0,6.0,7.0,79.0,79.0,10.7,58.9,31.8,18818.0


In [14]:
train_dataset.dtypes

carat      float64
cut          int64
color        int64
clarity      int64
depth      float64
table      float64
x          float64
y          float64
z          float64
price        int64
dtype: object

In [15]:
columnas = [a for a in train_dataset.columns if a not in ['price','size','prize-to-size']]
X = train_dataset[columnas]
y = train_dataset['price']

In [17]:
X=pd.get_dummies(X)

In [18]:
model = RandomForestRegressor(criterion="mse",max_features="auto",oob_score=False,n_estimators=750,n_jobs=-1)
for i in list(range(0,5)):
    X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)
    model.fit(X_train,y_train)
    y_pred= model.predict(X_test)
    print(f"1 expvariance ", round(explained_variance_score(y_test,y_pred),3))
    print(f"1 maxerr ", round(max_error(y_test,y_pred),3))
    print(f"1 meanabserr ", round(mean_absolute_error(y_test,y_pred),3))
    print(f"1 meansqr ", round(mean_squared_error(y_test,y_pred),3))
    print(f"1 root meansqr ", round(sqrt(mean_squared_error(y_test,y_pred)),3))
    print(f"1 meanabserr ", round(median_absolute_error(y_test,y_pred),3))
    print(f"1 r2-score ", round(r2_score(y_test,y_pred),3))
    print("-------------------")


1 expvariance  0.979
1 maxerr  9207.587
1 meanabserr  283.569
1 meansqr  337800.509
1 root meansqr  581.206
1 meanabserr  103.131
1 r2-score  0.979
-------------------
1 expvariance  0.979
1 maxerr  10036.049
1 meanabserr  282.024
1 meansqr  348728.937
1 root meansqr  590.533
1 meanabserr  99.061
1 r2-score  0.979
-------------------
1 expvariance  0.98
1 maxerr  6557.191
1 meanabserr  270.204
1 meansqr  314361.399
1 root meansqr  560.679
1 meanabserr  94.561
1 r2-score  0.98
-------------------
1 expvariance  0.98
1 maxerr  9264.176
1 meanabserr  269.214
1 meansqr  311326.882
1 root meansqr  557.967
1 meanabserr  99.0
1 r2-score  0.98
-------------------
1 expvariance  0.979
1 maxerr  8847.581
1 meanabserr  279.143
1 meansqr  345782.174
1 root meansqr  588.032
1 meanabserr  99.077
1 r2-score  0.979
-------------------


In [19]:
pred_test=pd.read_csv('./input/predict.csv',encoding='latin-1')

In [20]:
pred_test = pred_test.set_index('id', drop=True)
pred_test1 = pred_test.replace({'cut': labels_cut})
pred_test1 = pred_test1.replace({'color': labels_color})
pred_test1 = pred_test1.replace({'clarity': labels_clarity})
pred_test1.head()

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,0.33,2,5,0,62.0,58.0,4.44,4.46,2.76
1,1.21,2,0,6,62.4,58.0,6.77,6.83,4.24
2,1.06,2,0,5,59.3,60.0,6.64,6.71,3.96
3,0.36,0,1,1,61.4,57.0,4.64,4.61,2.54
4,0.70,0,1,3,62.3,54.0,5.67,5.72,3.55


In [21]:
y_pred =model.predict(pred_test1)
print(y_pred)
print(y_pred.shape)

[ 776.37866667 6021.76133333 5840.12355556 ... 2446.08       1287.604
 1059.56      ]
(13485,)


In [22]:
submission=pd.DataFrame(data=y_pred).reset_index()
submission=submission.rename(columns={"index":"id",0: "price"})
submission = submission.set_index('id', drop=True)

submission.head()

,price
id,
0,776.378667
1,6021.761333
2,5840.123556
3,1190.950667
4,3248.002667


In [23]:
submission.to_csv('./output/submit_10.csv')